In [2]:
import pandas as pd
import numpy as np

# 1) อ่านไฟล์ csv
df = pd.read_csv(r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\unknow-m_1d.csv")  # ใส่ path ของไฟล์ CSV ต้นฉบับ

# 2) แปลง timestamp เป็น datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# 3) กำหนดเกณฑ์ในการตัด outlier (3 standard deviations)
pm2_5_mean = df['pm_2_5'].mean()
pm2_5_std = df['pm_2_5'].std()
threshold_high_pm2_5 = pm2_5_mean + (3 * pm2_5_std)
threshold_low_pm2_5  = pm2_5_mean - (3 * pm2_5_std)

pm10_mean = df['pm_10'].mean()
pm10_std  = df['pm_10'].std()
threshold_high_pm10 = pm10_mean + (3 * pm10_std)
threshold_low_pm10  = pm10_mean - (3 * pm10_std)

# 4) กรองแถวที่อยู่ในช่วง threshold (ไม่เป็น outlier)
df_clean = df[
    (df['pm_2_5'] >= threshold_low_pm2_5) & (df['pm_2_5'] <= threshold_high_pm2_5) &
    (df['pm_10']  >= threshold_low_pm10)  & (df['pm_10']  <= threshold_high_pm10)
]

# 5) ลบแถวที่ pm_2_5 หรือ pm_10 เป็นศูนย์ (ถ้าถือว่าค่า 0 เกิดจากเซนเซอร์ผิดพลาด)
df_clean = df_clean[(df_clean['pm_2_5'] > 0) & (df_clean['pm_10'] > 0)]

# 6) จัดเรียงตาม timestamp และ reset index
df_clean = df_clean.sort_values(by='timestamp').reset_index(drop=True)

# 7) บันทึกไฟล์ CSV ใหม่
df_clean.to_csv("unknow-m_1d_clean.csv", index=False)


In [3]:
import pandas as pd
import numpy as np

# 1) อ่านไฟล์ต้นฉบับ
df = pd.read_csv(r"C:\Users\snpdp\pm2.5-Forecast-Dashboard\Data_csv\merge_data\unknow-m_1d.csv")

# 2) แปลง timestamp เป็น datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# --(Data Cleaning)-------------------------------------------
# 2.1) ตัด Outliers pm_2_5 และ pm_10 (3 standard deviations)
pm2_5_mean = df['pm_2_5'].mean()
pm2_5_std  = df['pm_2_5'].std()
threshold_high_pm2_5 = pm2_5_mean + (3 * pm2_5_std)
threshold_low_pm2_5  = pm2_5_mean - (3 * pm2_5_std)

pm10_mean = df['pm_10'].mean()
pm10_std  = df['pm_10'].std()
threshold_high_pm10 = pm10_mean + (3 * pm10_std)
threshold_low_pm10  = pm10_mean - (3 * pm10_std)

df_clean = df[
    (df['pm_2_5'] >= threshold_low_pm2_5) & (df['pm_2_5'] <= threshold_high_pm2_5) &
    (df['pm_10']  >= threshold_low_pm10)  & (df['pm_10']  <= threshold_high_pm10)
]

# 2.2) ลบแถวที่ pm_2_5 หรือ pm_10 เป็น 0
df_clean = df_clean[(df_clean['pm_2_5'] > 0) & (df_clean['pm_10'] > 0)]

# ------------------------------------------------------------

# 3) สร้างคอลัมน์ date โดยดึงเฉพาะ 'ปี-เดือน-วัน' ออกมา
df_clean['date'] = df_clean['timestamp'].dt.date

# 4) รวมข้อมูลซ้ำในวันเดียวกันด้วยการ group by
#    ตัวอย่างนี้จะหาค่าเฉลี่ย (mean) ของแต่ละคอลัมน์
df_daily = df_clean.groupby('date').agg({
    'pm_2_5': 'mean',
    'pm_10': 'mean',
    'humidity': 'mean',
    'temperature': 'mean'
})

# ถ้าอยากได้ค่าอื่นๆ เพิ่ม ก็สามารถเพิ่มลงไปใน .agg() ได้ เช่น
# 'pm_2_5': ['mean','max','min']

# 5) รีเซ็ตอินเด็กซ์เพื่อให้ 'date' กลายเป็นคอลัมน์ปกติ
df_daily = df_daily.reset_index()

# 6) บันทึกลงไฟล์ใหม่ (เหลือแค่ 1 แถว/1 วัน)
df_daily.to_csv("unknow-m_1d_clean_daily.csv", index=False)

print(df_daily.head())


         date     pm_2_5      pm_10   humidity  temperature
0  2021-02-12  18.155954  20.561082  83.780920    27.690057
1  2021-02-13  15.427183  17.448686  83.710912    27.675274
2  2021-02-14  27.776937  31.856245  79.459665    27.677253
3  2021-02-15  32.227238  37.315597  79.789949    27.440677
4  2021-02-16  34.262700  39.958193  79.357195    27.287088
